https://www.tensorflow.org/versions/r0.10/tutorials/word2vec/index.html#motivation-why-learn-word-embeddings





### Recurrent Neural Networks
Recurrent Neural Networks, 이하 RNN는 다음과 같은 구조를 가진 모델이다. RNN은 자기자신을 향하는 weight를 이용해 데이터간의 시간관계를 학습할 수 있다. 이러한 문제들을 시계열 학습이라고 부르며, 기존에 널리 쓰이던 Hidden Markov Model을 뉴럴넷을 이용해 구현했다고 볼 수 있다.




<img src  = "SimpleRNN01.png">
(출처: https://raw.githubusercontent.com/peterroelants/peterroelants.github.io/master/notebooks/RNN_implementation/img/SimpleRNN01.png)

위의 구조는 1개의 Recurrent weight를 가진 hidden node이다. 이러한 hidden node들이 여러개를 모여 1개의 RNN layer를 형성하고, 이것이 다시 deep 하게 쌓이는 모델 또한 가능하다.(그러나 RNN은 deep 하게 쌓을 경우 학습이 쉽지 않다.)

RNN의 경우 MLP나 CNN에 비해서 구현이 다소 복잡하다. 따라서 RNN은 skflow 라는 TensorFlow 공식 wrapping library를 활용해서 구현해보자.

https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/learn/python/learn

Skflow는 사용자가 최소한의 노력으로, 매우 추상화된 함수들을 사용해 deep learning 모델을 구축하고 학습할 수 있게 도와주는 역할을 한다.


<br>아래의 예제 코드를 실행시키기 위해서는 리눅스 shell에서 다음의 명령어를 실행해 필요한 python package를 설치해야한다.
####### pip install pandas

<img src  = "word_embedding.png" width='600'>
(출처: http://sebastianruder.com/word-embeddings-1/)

In [1]:
import numpy as np
from sklearn import metrics
import pandas

import tensorflow as tf
from tensorflow.contrib import learn

### Downloads Training data, unpacks and reads DBpedia dataset.
dbpedia = learn.datasets.load_dataset('dbpedia')
X_train, y_train = pandas.DataFrame(dbpedia.train.data)[1], pandas.Series(dbpedia.train.target)
X_test, y_test = pandas.DataFrame(dbpedia.test.data)[1], pandas.Series(dbpedia.test.target)

### Process vocabulary
MAX_DOCUMENT_LENGTH = 10

vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
X_train = np.array(list(vocab_processor.fit_transform(X_train)))
X_test = np.array(list(vocab_processor.transform(X_test)))

n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

### Models
EMBEDDING_SIZE = 50

# Customized function to transform batched X into embeddings
def input_op_fn(X):
    # Convert indexes of words into embeddings.
    # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
    # maps word indexes of the sequence into [batch_size, sequence_length,
    # EMBEDDING_SIZE].
    word_vectors = learn.ops.categorical_variable(X, n_classes=n_words,
        embedding_size=EMBEDDING_SIZE, name='words')
    # Split into list of embedding per word, while removing doc length dim.
    # word_list results to be a list of tensors [batch_size, EMBEDDING_SIZE].
    word_list = learn.ops.split_squeeze(1, MAX_DOCUMENT_LENGTH, word_vectors)
    return word_list

# Single direction GRU with a single layer
classifier = learn.TensorFlowRNNClassifier(rnn_size=EMBEDDING_SIZE, 
    n_classes=15, cell_type='gru', input_op_fn=input_op_fn,
    num_layers=1, bidirectional=False, sequence_length=None,
    batch_size = 32, steps=1000, optimizer='Adam', learning_rate=0.01, continue_training=True)

classifier.sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))


# train for 1000 steps & predict on test set.
classifier.fit(X_train, y_train, logdir='/tmp/tf_examples/word_rnn')
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print('Accuracy: {0:f}'.format(score))

classifier.sess.close()

Instructions for updating:
Please use tf.unpack instead.
Instructions for updating:
Please use tf.unpack instead.
Instructions for updating:
Please use tf.unpack instead.
Instructions for updating:
Please use tf.unpack instead.


Total words: 7664
Accuracy: 0.485714
